# InSAR Time Series Analysis using MintPy and ARIA products

**Author:** Zhang Yunjun, Heresh Fattahi, modified for NISAR Solid Earth Science Team by David Bekaert

The Miami Insar Timeseries software in PYthon (MintPy) is an open-source package for InSAR time-series analysis. Mintpy is compatible with the outputs from the ARIA-tools **ariaTSsetup.py** program, and can be used to estimate ground displacement time-series. In the example below, we will demonstrate a time-series derived from ARIA standard InSAR products over the San Francisco bay.

<div class="alert alert-warning">
<b>To safe time, we have already ran the ARIA-tools package to download and prepared the time-series stack:</b> 
    
ariaDownload.py -b '37.25 38.1 -122.6 -121.75' --track 42

ariaTSsetup.py -f 'products/*.nc' -b '37.25 38.1 -122.6 -121.75' --mask Download

<b>We have also loaded the ariaTSsetup.py outputs into the Mintpy format:</b>
    
prep_aria.py -s ../04_ariaTSprep/stack/ -i ../04_ariaTSprep/incidenceAngle/20150605_20150512.vrt -d ../04_ariaTSprep/DEM/SRTM_3arcsec.dem -a ../04_ariaTSprep/azimuthAngle/20150605_20150512.vrt --water-mask ../04_ariaTSprep/mask/watermask.msk
</div>

# 0. Notebook setup

The cell below is required to be ran each time the notebook is started and ensures correct set-up of the notebook.

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from mintpy import view, tsview, plot_network, plot_transection, plot_coherence_matrix

# define the work directory
work_dir = os.path.abspath(os.getcwd())

#The Path to the folder where ARIA data after running ariaTSsetup.py is stored
aria_dir = os.path.join(os.getenv("HOME"),'work/04_ariaTSprep')

print("Work directory: ", work_dir)
print("The path to the directory with ARIA data after running ariaTSsetup:", aria_dir)

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)
    print('Create directory: {}'.format(work_dir))

print('Go to work directory: {}'.format(work_dir))
os.chdir(work_dir)

# 1. smallbaselineApp.py overview

The MintPy **smallbaselineApp.py** application provides a workflow to invert a stack of unwrapped interferograms and apply different (often optional) corrections to obtain ground displacement timeseries. A detailed overview of the options can be retrieved by involking the help option:

In [ ]:
!smallbaselineApp.py --help

## 1.2 Configuring processing parameters

The processing parameters for the **smallbaselineApp.py** are controlled through a configuration file. If no file is provided the default [smallbaselineApp.cfg](https://github.com/insarlab/MintPy/blob/master/mintpy/defaults/smallbaselineApp.cfg) configuration is used. Here we use `SanFranSenDT42.txt`, which already constains selective, manually modified configuration parameters.

# 2. Small Baseline Time Series Analysis

## 2.1. Loading ARIA data into MintPy

Use the **prep_aria.py** program to load the ARIA products into MintPy. The **prep_aria.py program takes the outputs from ariaTSsetup.py and creates HDF5 files** which contains the stack of unwrapped interferograms, coherence, connected components and required metdata which are **used as inputs to MintPy**. 

This creates the "inputs" directory containing two HDF5 files:
- ifgramStack.h5: This file contains 6 dataset cubes (e.g. unwrapped phase, coherence, connected components etc.) and multiple metadata.  
- geometryGeo.h5: This file contains geometrical datasets (e.g., incidence/azimuth angle, masks, etc.). 

```!prep_aria.py -s ../04_ariaTSprep/stack/ -i ../04_ariaTSprep/incidenceAngle/20150605_20150512.vrt -d ../04_ariaTSprep/DEM/SRTM_3arcsec.dem -a ../04_ariaTSprep/azimuthAngle/20150605_20150512.vrt --water-mask ../04_ariaTSprep/mask/watermask.msk```

```
extract metadata from /Users/dbekaert/Software/SET_demo/04_ariaTSprep/stack/unwrapStack.vrt
--------------------------------------------------
create HDF5 file /Users/dbekaert/Software/SET_demo/mintPyTS_test2/inputs/geometryGeo.h5 with w mode
create dataset: height                    of <class 'numpy.float32'>   in size of (1020, 1020)
create dataset: incidenceAngle            of <class 'numpy.float32'>   in size of (1020, 1020)
create dataset: slantRangeDistance        of <class 'numpy.float32'>   in size of (1020, 1020)
create dataset: azimuthAngle              of <class 'numpy.float32'>   in size of (1020, 1020)
create dataset: waterMask                 of <class 'numpy.bool_'>     in size of (1020, 1020)
close HDF5 file /Users/dbekaert/Software/SET_demo/mintPyTS_test2/inputs/geometryGeo.h5
--------------------------------------------------
writing data to HDF5 file /Users/dbekaert/Software/SET_demo/mintPyTS_test2/inputs/geometryGeo.h5 with a mode ...
finished writing to HD5 file: /Users/dbekaert/Software/SET_demo/mintPyTS_test2/inputs/geometryGeo.h5
--------------------------------------------------
create HDF5 file /Users/dbekaert/Software/SET_demo/mintPyTS_test2/inputs/ifgramStack.h5 with w mode
create dataset: bperp                     of <class 'numpy.float32'>   in size of (443,)
create dataset: coherence                 of <class 'numpy.float32'>   in size of (443, 1020, 1020)
create dataset: connectComponent          of <class 'numpy.int16'>     in size of (443, 1020, 1020)
create dataset: date                      of |S8                       in size of (443, 2)
create dataset: dropIfgram                of <class 'numpy.bool_'>     in size of (443,)
create dataset: unwrapPhase               of <class 'numpy.float32'>   in size of (443, 1020, 1020)
close HDF5 file /Users/dbekaert/Software/SET_demo/mintPyTS_test2/inputs/ifgramStack.h5
--------------------------------------------------
opening unwrapStack.vrt, cohStack.vrt, connCompStack.vrt with gdal ...
grab NoDataValue for unwrapPhase:      0.0   and convert to 0.
grab NoDataValue for coherence:        0.0   and convert to 0.
grab NoDataValue for connectComponent: -1.0  and convert to 0.
writing data to HDF5 file /Users/dbekaert/Software/SET_demo/mintPyTS_test2/inputs/ifgramStack.h5 with a mode ...
[==================================================] 20190912_20190924   77s /     1s
finished writing to HD5 file: /Users/dbekaert/Software/SET_demo/mintPyTS_test2/inputs/ifgramStack.h5
```

<div class="alert alert-info">
<b>info.py :</b> 
To get general infomation about a MintPy product, run info.py on the file.   
</div>

In [ ]:
!info.py inputs/ifgramStack.h5

In [ ]:
!info.py inputs/geometryGeo.h5

## 2.2. Plotting the interferogram network

Running **plot_network.py** gives an overview of the network and the average coherence of the stack. The program creates multiple files as follows:
- ifgramStack_coherence_spatialAvg.txt: Contains interferogram dates, average coherence temporal and spatial baseline separation.
- Network.pdf: Displays the network of interferograms on time-baseline coordinates, colorcoded by avergae coherence of the interferograms. 
- CoherenceMatrix.pdf shows the avergae coherence pairs between all available pairs in the stack.

In [ ]:
!smallbaselineApp.py SanFranSenDT42.txt  --dostep modify_network
plot_network.main(['inputs/ifgramStack.h5'])


## 2.3.  Mask generation

Mask files can be can be used to mask pixels in the time-series processing. Below we generate a mask file based on the connected components, which is a metric for unwrapping quality.

In [ ]:
!generate_mask.py  inputs/ifgramStack.h5  --nonzero  -o maskConnComp.h5  --update
view.main(['maskConnComp.h5'])

## 2.4. Inverting of the Small Baseline network

In the next step we invert the network of differential unwrapped interferograms to estimate the time-series of unwrapped phase with respect to a reference acquisition date. By default mintpy selects the first acquisition. The estimated time-series is converted to distance change from radar to target and is provided in meters. 

In [ ]:
!smallbaselineApp.py SanFranSenDT42.txt --dostep reference_point
!smallbaselineApp.py SanFranSenDT42.txt --dostep invert_network

The timeseries file contains three datasets:
- the "time-series" which is the interferometric range change for each acquisition relative to the reference acquisition,
- the "date" dataset which contains the acquisition date for each acquisition,
- the "bperp" dataset which contains the timeseries of the perpendicular baseline.  

## 2.5. Estimating the long-term velocity rate

The ground deformation caused by many geophysical or anthropogenic processes are linear at first order approximation. Therefore it is common to estimate the rate of the ground deformation which is the slope of linear fit to the time-series. 

In [ ]:
!smallbaselineApp.py SanFranSenDT42.txt --dostep velocity

In [ ]:
scp_args = 'velocity.h5 velocity -v -1 1'
view.main(scp_args.split())

<div class="alert alert-info">
<b>Note :</b> 
Negative values indicates that target is moving away from the radar (i.e., Subsidence in case of vertical deformation).
Positive values indicates that target is moving towards the radar (i.e., uplift in case of vertical deformation).
</div>

Obvious features in the estimated velocity map:

1) The general pattern of displacement is consistent with tectonic setting of the region. Pacific plate is moving north-west with respect to North american plate. The satellites is on a descending track and the estimated displacement shows the blue region is moving away from the red region. 

2) The magnitude of the relative movement (blue region relative to red region) is about ~15 mm/yr in radar Line-Of-Sight (LOS) direction which is consistent with ~40 mm/yr horizontal movements of pacific relative to north america.

3) The estimated velocity shows a linear feature almost aligned with the south-east north-west diagonal of the map. This linear feature shows the aseismic fault creep on Hayward fault. 

4) Further north and around latitude 38N another linear feature represents Concord fault parallel to Hayward fault.

5) Around latitude 37.9 N, a east-west linear discontinuity is evident. This is most likely caused by missing bursts in some interferograms. 

6) The red region at the south-east corenr (around 37.35 N, 121.9W) seems to be a hydrological signal showing ground uplift caused by acquifer recharge.

7) The block box at 37.7N, 122.3W is the reference pixel for this map. 

## 2.6. Velocity error analysis

The estimated velocity also comes with an expression of unecrtainty which is simply based on the goodness of fit while fitting a linear model to the time-series. This quantity is saved in "velocity.h5" under the velocityStd dataset. 

**Mintpy supports additional corrections in its processing not included in this demo:**
- Unwrapping error correction
- Tropospheric delay correction
- deramping
- Topographic residual correction
- Residual RMS for noise evaluation
- Changing the reference date

In [ ]:
scp_args = 'velocity.h5 velocityStd -v 0 0.2'
view.main(scp_args.split())

# 3. Plotting a Fault Creep Transect 

In [ ]:
scp_args = 'velocity.h5 --start-lalo 37.5307 -122.3631 --end-lalo 37.9828 -121.7707 '
scp_args = 'velocity.h5 --start-lalo 37.6523 -122.1163 --end-lalo 37.7047 -122.0497 '
plot_transection.main(scp_args.split())

!smallbaselineApp.py smallbaselineApp.cfg --dostep google_earth

The transect across Hayward fault shows ~2.5 mm/yr fault creep in LOS direction.

# 4. Validating InSAR with GPS

MintPy can automatically download GPS data over the region of interest from Nevada Geodetic Laboratory at University of Nevada, Reno. 

Thew following command plots add dowloads GPS data, project the components to InSAR LOS direction and plot them on the InSAR velocity map:   

In [ ]:
scp_args = 'velocity.h5 velocity --show-gps --ref-gps P225 --gps-comp enu2los --gps-label --fontsize 8'
view.main(scp_args.split())

# Reference material

- Original Notebook withe detailed description by Yunjun and Fattahi at: https://nbviewer.jupyter.org/github/insarlab/MintPy-tutorial/blob/master/smallbaselineApp_aria.ipynb

- Mintpy reference: *Yunjun, Z., H. Fattahi, F. Amelung (2019), Small baseline InSAR time series analysis: unwrapping error correction and noise reduction, preprint doi:[10.31223/osf.io/9sz6m](https://eartharxiv.org/9sz6m/).*

- University of Miami online time-series viewer: https://insarmaps.miami.edu/

- Mintpy Github repository: https://github.com/insarlab/MintPy

- ARIA-tools Github Repository: https://github.com/aria-tools/ARIA-tools